# Valuing European-style Swaptions with QL Example

We value a European swaption using Black's model and try to replicate a QL example at http://khandrikacm.blogspot.com/2014/03/european-style-interest-rate-swaption.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.products.rates import *
from financepy.utils import *
from financepy.market.curves import *
from financepy.models.black import Black

## Building a Ibor Discount Curve

In [3]:
value_dt = Date(28, 2, 2014)
settle_dt = Date(4, 3, 2014)

We have a vector of dates and zero rates

In [4]:
depo_dcc_type = DayCountTypes.THIRTY_E_360_ISDA
depos = []

depo = IborDeposit(settle_dt, "1W", 0.0023, depo_dcc_type); depos.append(depo)
depo = IborDeposit(settle_dt, "1M", 0.0023, depo_dcc_type); depos.append(depo)
depo = IborDeposit(settle_dt, "3M", 0.0023, depo_dcc_type); depos.append(depo)
depo = IborDeposit(settle_dt, "6M", 0.0023, depo_dcc_type); depos.append(depo)

# No convexity correction provided so I omit interest rate futures

swaps = []
swap_type = SwapTypes.PAY
fixed_dcc_type = DayCountTypes.ACT_365F
fixed_freq_type = FrequencyTypes.SEMI_ANNUAL

swap = IborSwap(settle_dt, "3Y", swap_type, 0.00790, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "4Y", swap_type, 0.01200, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "5Y", swap_type, 0.01570, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "6Y", swap_type, 0.01865, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "7Y", swap_type, 0.02160, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "8Y", swap_type, 0.02350, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "9Y", swap_type, 0.02540, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "10Y", swap_type, 0.0273, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "15Y", swap_type, 0.0297, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "20Y", swap_type, 0.0316, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "25Y", swap_type, 0.0335, fixed_freq_type, fixed_dcc_type); swaps.append(swap)
swap = IborSwap(settle_dt, "30Y", swap_type, 0.0354, fixed_freq_type, fixed_dcc_type); swaps.append(swap)

libor_curve = IborSingleCurve(value_dt, depos, [], swaps)

In [5]:
exercise_date = settle_dt.add_tenor("5Y")
swap_maturity_dt = exercise_date.add_tenor("5Y")

swap_fixed_cpn = 0.040852
swapFixedFrequencyType = FrequencyTypes.SEMI_ANNUAL
swapFixedDayCountType = DayCountTypes.THIRTY_E_360_ISDA

swapFloatFrequencyType = FrequencyTypes.QUARTERLY
swapFloatDayCountType = DayCountTypes.ACT_360

swapNotional = ONE_MILLION
swap_type = SwapTypes.PAY
cal_type = CalendarTypes.TARGET
bd_type = BusDayAdjustTypes.MODIFIED_FOLLOWING
dg_type = DateGenRuleTypes.BACKWARD

In [6]:
swaption = IborSwaption(settle_dt,
                            exercise_date,
                            swap_maturity_dt,
                            swap_type,
                            swap_fixed_cpn,
                            swapFixedFrequencyType,
                            swapFixedDayCountType,
                            swapNotional,
                            swapFloatFrequencyType,
                            swapFloatDayCountType,
                            cal_type,
                            bd_type,
                            dg_type)

In [7]:
print(swaption)

OBJECT TYPE: IborSwaption
SETTLEMENT DATE: 04-MAR-2014
EXERCISE DATE: 04-MAR-2019
SWAP FIXED LEG TYPE: SwapTypes.PAY
SWAP MATURITY DATE: 04-MAR-2024
SWAP NOTIONAL: 1000000
FIXED cpn: 4.0852
FIXED FREQUENCY: FrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: FrequencyTypes.QUARTERLY
FLOAT DAY COUNT: DayCountTypes.ACT_360



In [8]:
model = Black(0.1533)

In [9]:
swaption.value(settle_dt, libor_curve, model)

np.float64(24666.078516312486)

This is close to a value of 23162 using QL

## Internals

In [10]:
print(swaption)

OBJECT TYPE: IborSwaption
SETTLEMENT DATE: 04-MAR-2014
EXERCISE DATE: 04-MAR-2019
SWAP FIXED LEG TYPE: SwapTypes.PAY
SWAP MATURITY DATE: 04-MAR-2024
SWAP NOTIONAL: 1000000
FIXED cpn: 4.0852
FIXED FREQUENCY: FrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: FrequencyTypes.QUARTERLY
FLOAT DAY COUNT: DayCountTypes.ACT_360
PV01: 4.160189726243439
FWD SWAP RATE: 4.148986244425445
FWD DF TO EXPIRY: 0.923742186114948


We can see that the forward swap rate almost equals the fixed cpn. The underlying swap is close to being ATM forward.

In [11]:
swaption.print_swap_fixed_leg()

START DATE: 04-MAR-2019
MATURITY DATE: 04-MAR-2024
COUPON (%): 4.0852
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA

PAYMENTS VALUATION:
PAY_NUM | PAY_dt      | NOTIONAL | RATE   | PMNT     | DF     | PV       | CUM_PV   
--------+-------------+----------+--------+----------+--------+----------+----------
      1 | 04-SEP-2019 |  1000000 | 4.0852 |  20426.0 |  0.908 | 18546.05 |  18546.05
      2 | 04-MAR-2020 |  1000000 | 4.0852 |  20426.0 | 0.8926 | 18232.34 |  36778.39
      3 | 04-SEP-2020 |  1000000 | 4.0852 |  20426.0 | 0.8744 | 17861.51 |  54639.89
      4 | 04-MAR-2021 |  1000000 | 4.0852 |  20426.0 | 0.8569 | 17503.38 |  72143.27
      5 | 06-SEP-2021 |  1000000 | 4.0852 | 20652.96 | 0.8404 | 17356.15 |  89499.42
      6 | 04-MAR-2022 |  1000000 | 4.0852 | 20199.04 | 0.8248 |  16659.4 | 106158.82
      7 | 05-SEP-2022 |  1000000 | 4.0852 | 20539.48 | 0.8072 | 16578.67 | 122737.49
      8 | 06-MAR-2023 |  1000000 | 4.0852 | 20539.48 | 0.7902 |

In [12]:
swaption.print_swap_float_leg()

START DATE: 04-MAR-2019
MATURITY DATE: 04-MAR-2024
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.ACT_360

PAYMENTS VALUATION:
PAY_NUM | PAY_dt      | NOTIONAL | IBOR   | PMNT     | DF     | PV      | CUM_PV   
--------+-------------+----------+--------+----------+--------+---------+----------
      1 | 04-JUN-2019 |  1000000 | 3.4642 |  8853.04 | 0.9158 | 8107.92 |   8107.92
      2 | 04-SEP-2019 |  1000000 | 3.4677 |  8861.85 |  0.908 | 8046.23 |  16154.15
      3 | 04-DEC-2019 |  1000000 | 3.4298 |  8669.84 | 0.9002 | 7804.97 |  23959.12
      4 | 04-MAR-2020 |  1000000 | 3.4233 |  8653.37 | 0.8926 | 7724.04 |  31683.16
      5 | 04-JUN-2020 |  1000000 | 4.1266 | 10545.87 | 0.8835 | 9317.17 |  41000.33
      6 | 04-SEP-2020 |  1000000 | 4.1353 | 10567.99 | 0.8744 | 9241.18 |   50241.5
      7 | 04-DEC-2020 |  1000000 | 4.0901 | 10338.92 | 0.8656 | 8949.36 |  59190.86
      8 | 04-MAR-2021 |  1000000 | 4.0526 | 10131.56 | 0.8569 |  8681.9 |  67872.76
 

Copyright (c) 2020 Dominic O'Kane